In [1]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from skimage import morphology
import os
import random
from skimage.measure import label,regionprops

In [2]:
#root = "Z:/Projectline_modelling_lung_cancer/Nii_Processed/"
root = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CT_ITV_GTV_XBP_Nii_Processed/"

In [3]:
PxList_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/PxSelectionforDCM2Nii.txt"
with open(PxList_path, 'r') as file:
    PxList = [line.rstrip() for line in file]
print(len(PxList))

861


In [5]:
def LookFiles_Snapshot(folder_path):
    planCT_list,planCTLung_list = ([],[])
    bp50_list, bp50Lung_list = ([],[])
    gtvP_list, gtvN_list = ([],[])
    itvP_list, itvN_list = ([],[])
    for root,dirs,files in os.walk(folder_path):
        for f in files:
            if "PlanCT_CTProcessed.nii" in f:
                planCT_list.append(os.path.join(folder_path,f))
            if "PlanCT_LungProcessed.nii" in f:
                planCTLung_list.append(os.path.join(folder_path,f))
            if "bp50_CTProcessed.nii" in f:
                bp50_list.append(os.path.join(folder_path,f))
            if "bp50_LungProcessed.nii" in f:
                bp50Lung_list.append(os.path.join(folder_path,f))
            if "GTVn_GTProcessed.nii" in f:
                gtvN_list.append(os.path.join(folder_path,f))
            if "GTVp_GTProcessed.nii" in f:
                gtvP_list.append(os.path.join(folder_path,f))
            if "ITVn_GTProcessed.nii" in f:
                itvN_list.append(os.path.join(folder_path,f))
            if "ITVp_GTProcessed.nii" in f:
                itvP_list.append(os.path.join(folder_path,f))
    if len(bp50_list)==0: 
        for root,dirs,files in os.walk(folder_path):
            for f in files:
                if "bp60_CTProcessed.nii" in f:
                    bp50_list.append(os.path.join(folder_path,f))
                if "bp60_LungProcessed.nii" in f:
                    bp50Lung_list.append(os.path.join(folder_path,f))
    if len(bp50_list)==0: 
        for root,dirs,files in os.walk(folder_path):
            for f in files:
                if "bp40_CTProcessed.nii" in f:
                    bp50_list.append(os.path.join(folder_path,f))
                if "bp40_LungProcessed.nii" in f:
                    bp50Lung_list.append(os.path.join(folder_path,f))
    
    #print("Len of Files",len(planCT_list),len(planCTLung_list),len(bp50_list),len(bp50Lung_list),len(gtv_list),len(itv_list))


    return planCT_list,planCTLung_list,bp50_list,bp50Lung_list,itvP_list,itvN_list,gtvP_list, gtvN_list
    

In [6]:
def CheckBlobs(struct_np):
    labeled_np = label(struct_np)
    regions_3d = regionprops(labeled_np)
    numBlobs = len(regions_3d)
    list_of_volume = [region.area_filled for region in regions_3d]
    return numBlobs,list_of_volume

In [7]:
def largestRatio(struct_np):
    ratioVal_x,ratioVal_y,ratioVal_z = 0,0,0
    listRatios_x,listRatios_y,listRatios_z = [],[],[]
    #Z
    maxRatio = 0
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i])>0:
            currPosSum = np.sum(struct_np[:,:,i])
            currTotSum = struct_np.shape[0]*struct_np.shape[1]
            currRatio = currPosSum/currTotSum
            listRatios_z.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_z = maxRatio
    maxSlide_z = maxSlide
    #Y
    maxRatio = 0
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:])>0:
            currPosSum = np.sum(struct_np[:,i,:])
            currTotSum = struct_np.shape[0]*struct_np.shape[2]
            currRatio = currPosSum/currTotSum
            listRatios_y.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_y = maxRatio
    maxSlide_y = maxSlide
    #X
    maxRatio = 0
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:])>0:
            currPosSum = np.sum(struct_np[i,:,:])
            currTotSum = struct_np.shape[1]*struct_np.shape[2]
            currRatio = currPosSum/currTotSum
            listRatios_x.append(currRatio)
            if currRatio>maxRatio:
                maxRatio = currRatio
                maxSlide = i
    ratioVal_x = maxRatio
    maxSlide_x = maxSlide

    return listRatios_x,listRatios_y,listRatios_z,ratioVal_x,ratioVal_y,ratioVal_z,maxSlide_x,maxSlide_y,maxSlide_z

In [8]:
def GetStructNP_ReturnRatios(Px,struct_np,classCurrStruct):

    TotSum = np.sum(struct_np)
    listOfShapes = [struct_np.shape[0],struct_np.shape[1],struct_np.shape[2]]
    if TotSum>0:

        listRatios_x,listRatios_y,listRatios_z,ratioVal_x,ratioVal_y,ratioVal_z,maxSlide_x,maxSlide_y,maxSlide_z = largestRatio(struct_np)

        MaxRatios = [ratioVal_x,ratioVal_y,ratioVal_z]
        AvgRatios = [np.mean(listRatios_x),np.mean(listRatios_y),np.mean(listRatios_z)]
        MedianRatios = [np.median(listRatios_x),np.median(listRatios_y),np.median(listRatios_z)]

        #numBlobs,volumeBlobs = CheckBlobs(struct_np)

        return classCurrStruct(Px,TotSum,listOfShapes,MaxRatios,AvgRatios,MedianRatios)
    else:
        return classCurrStruct(Px,TotSum,listOfShapes,None,None,None)


In [9]:
class currStruct:
    def __init__(self,Px,TotSum,Shapes, MaxRatio, AvgRatio,MedRatio):
        self.Px = Px
        self.TotSum = TotSum
        self.Shapes = Shapes
        self.MaxRatio = MaxRatio
        self.AvgRatio = AvgRatio
        self.MedRatio = MedRatio
        

In [10]:
def GetMeanValuesofList(listClassGTVP):
    attrSumX,attrSumY,attrSumZ = 0,0,0
    listShapesX,listShapesY,listShapesZ = [],[],[]
    for currObj in listClassGTVP:
        if currObj.AvgRatio is not None:
            attrSumX+=currObj.AvgRatio[0]
            attrSumY+=currObj.AvgRatio[1]
            attrSumZ+=currObj.AvgRatio[2]
        listShapesX.append(currObj.Shapes[0])
        listShapesY.append(currObj.Shapes[1])
        listShapesZ.append(currObj.Shapes[2])
    
    AvgAttr = [attrSumX/len(listClassGTVP),attrSumY/len(listClassGTVP),attrSumZ/len(listClassGTVP)]
    listOfUnique = [np.unique(listShapesX),np.unique(listShapesY),np.unique(listShapesZ)]

    return AvgAttr,listOfUnique

In [11]:
listClassGTVP,listClassGTVN = [],[]
listClassITVP,listClassITVN = [],[]
okPx,errorPx = [],[]
countOkGTVPx=0
countOkITVPx=0

for Px in PxList:
    planCT_list,planCTLung_list,bp50_list,bp50Lung_list,itvP_list,itvN_list,gtvP_list, gtvN_list = LookFiles_Snapshot(root+Px)
    
    #ReadImages
    if False:
        ct_np  = nib.load(planCT_list[0]).get_fdata()
        lung_np  = nib.load(planCTLung_list[0]).get_fdata()
        bp50_np  = nib.load(bp50_list[0]).get_fdata()
        bp50Lung_np  = nib.load(bp50Lung_list[0]).get_fdata()
        #aveLung_filled = FixHoles(lung_np)
        #bp50Lung_filled = FixHoles(bp50Lung_np)

    if len(gtvP_list)>0 and len(gtvN_list)>0:
        gtvp_np  = nib.load(gtvP_list[0]).get_fdata()
        gtvn_np  = nib.load(gtvP_list[0]).get_fdata()
        listClassGTVP.append(GetStructNP_ReturnRatios(Px,gtvp_np,currStruct))
        listClassGTVN.append(GetStructNP_ReturnRatios(Px,gtvn_np,currStruct))
        countOkGTVPx+=1
    if len(itvP_list)>0 and len(itvN_list)>0:
        itvp_np  = nib.load(itvP_list[0]).get_fdata()
        itvn_np  = nib.load(itvN_list[0]).get_fdata()
        listClassITVP.append(GetStructNP_ReturnRatios(Px,itvp_np,currStruct))
        listClassITVN.append(GetStructNP_ReturnRatios(Px,itvn_np,currStruct))
        countOkITVPx+=1
    
AvgRatioGTVp,listOfUniqueGTVp = GetMeanValuesofList(listClassGTVP)
AvgRatioGTVn,listOfUniqueGTVn = GetMeanValuesofList(listClassGTVN)
AvgRatioITVp,listOfUniqueITVp = GetMeanValuesofList(listClassITVP)
AvgRatioITVn,listOfUniqueITVn = GetMeanValuesofList(listClassITVP)

print("GTV ok",countOkGTVPx,"ITV ok",countOkITVPx)
print("GTVp AvgRatio",AvgRatioGTVp,"GTVn AvgRatio",AvgRatioGTVn)
print("ITVp AvgRatio",AvgRatioITVp,"ITVn AvgRatio",AvgRatioITVn)

print("Shapes GTV",listOfUniqueGTVp)
print("Shapes ITV",listOfUniqueITVp)



In [ ]:
def Screenshot(ct_np,gtv_np,middle_x,middle_y,middle_z,contourBool=True,Px2Save=None,ImageType=None):
    minHU = -1024
    maxHU = 600
    savebool = True
    #root2save= "Z:/Projectline_modelling_lung_cancer/JPEGs/Processed/V1/"
    root2save= "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/JPEGs/Nii_Processed/"
    if Px2Save is not None:
        pathToSave = os.path.join(root2save,Px2Save)
        if not(os.path.exists(pathToSave)):
            os.mkdir(pathToSave)
    
    print(middle_x,middle_y,middle_z,Px2Save,ct_np.shape,ImageType,gtv_np.shape)
    
    clipped_image = np.clip(ct_np, minHU, maxHU)
    #middle_x,middle_y,middle_z = getMiddleSlices(gtv_np)
    plt.figure(figsize=(15, 15), dpi=150)#dpi=300)
    plt.imshow(clipped_image[:,:,middle_z],cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Z:"+str(middle_z))
    if contourBool:
        plt.contour(gtv_np[:,:,middle_z],linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Axial.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[:,middle_y,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Y:"+str(middle_y))
    if contourBool:
        plt.contour(np.rot90(gtv_np[:,middle_y,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Sagital.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[middle_x,:,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("X:"+str(middle_x))
    if contourBool:
        plt.contour(np.rot90(gtv_np[middle_x,:,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Coronal.jpeg")
        plt.close()

In [ ]:

def CountSlidesPerCategory(struct_np,tumorSlide_Tresh):
    #Z
    countTum_z = 0
    countSemi_z =0
    indices_Tum_z = []
    indices_Semi_z = []
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i])>0:
            currRatio = np.sum(struct_np[:,:,i])/(struct_np.shape[0]*struct_np.shape[1])
            if currRatio > tumorSlide_Tresh[-1]:
                countTum_z+=1
                indices_Tum_z.append(i)
            elif currRatio > 0:
                countSemi_z+=1
                indices_Semi_z.append(i)
    #Y
    countTum_y = 0
    countSemi_y =0
    indices_Tum_y= []
    indices_Semi_y = []
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:])>0:
            currRatio = np.sum(struct_np[:,i,:])/(struct_np.shape[0]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-2]:
                countTum_y+=1
                indices_Tum_y.append(i)
            elif currRatio > 0:
                countSemi_y+=1
                indices_Semi_y.append(i)
    #X
    countTum_x = 0
    countSemi_x =0
    indices_Tum_x= []
    indices_Semi_x = []
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:])>0:
            currRatio = np.sum(struct_np[i,:,:])/(struct_np.shape[1]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-3]:
                countTum_x+=1
                indices_Tum_x.append(i)
            elif currRatio > 0:
                countSemi_x+=1
                indices_Semi_x.append(i)

    
    #return countTum_x,countTum_y,countTum_z,countSemi_x,countSemi_y,countSemi_z,
    return indices_Tum_x,indices_Tum_y,indices_Tum_z,indices_Semi_x,indices_Semi_y,indices_Semi_z

In [ ]:
def FixHoles(struct_np):
    struct_np[struct_np>0] = 1
    struct_bool = morphology.binary_closing(struct_np.astype(bool))
    struct_filled = struct_bool.astype(np.uint8)
    return struct_filled

In [ ]:
factor=.3
GTVpSlide_AvgTresh = [AvgRatioGTVp[0]*factor,AvgRatioGTVp[1]*factor,AvgRatioGTVp[2]*factor]
GTVnSlide_AvgTresh = [AvgRatioGTVn[0]*factor,AvgRatioGTVn[1]*factor,AvgRatioGTVn[2]*factor]
ITVpSlide_AvgTresh = [AvgRatioITVp[0]*factor,AvgRatioITVp[1]*factor,AvgRatioITVp[2]*factor]
ITVnSlide_AvgTresh = [AvgRatioITVn[0]*factor,AvgRatioITVn[1]*factor,AvgRatioITVn[2]*factor]

In [ ]:
def SelectIndexAndScreenshot(Px,ContourType,indices_x,indices_y,indices_z,ct_np,struct_np):
    if len(indices_x)>0: 
        randomc_x = random.choice(indices_x)
        randomc_y = random.choice(indices_y)
        randomc_z = random.choice(indices_z)
        Screenshot(ct_np,struct_np,randomc_x,randomc_y,randomc_z,True,Px,ContourType)
    

In [ ]:
HR_GTVp_x,HR_GTVp_y,HR_GTVp_z = 0,0,0
HR_GTVn_x,HR_GTVn_y,HR_GTVn_z = 0,0,0
LR_GTVp_x,LR_GTVp_y,LR_GTVp_z = 0,0,0
LR_GTVn_x,LR_GTVn_y,LR_GTVn_z = 0,0,0

totHR_GTVp_x,totHR_GTVp_y,totHR_GTVp_z,totLR_GTVp_x,totLR_GTVp_y,totLR_GTVp_z = 0,0,0,0,0,0
totHR_GTVn_x,totHR_GTVn_y,totHR_GTVn_z,totLR_GTVn_x,totLR_GTVn_y,totLR_GTVn_z = 0,0,0,0,0,0

totHR_ITVp_x,totHR_ITVp_y,totHR_ITVp_z,totLR_ITVp_x,totLR_ITVp_y,totLR_ITVp_z = 0,0,0,0,0,0
totHR_ITVn_x,totHR_ITVn_y,totHR_ITVn_z,totLR_ITVn_x,totLR_ITVn_y,totLR_ITVn_z = 0,0,0,0,0,0
count =0

missingPlanCT_px = ['8034617', '1127507', '4809024', '772611', '6557018','8882956', '986614','5474853', '340941', '9034825', '2741303', '2659861', '6958675', '2181664', '957793']
diffShape = ['3327503','2126847','699039','750710','9236751']
corrupt = missingPlanCT_px+diffShape

for Px in PxList:

    if np.any(Px  in corrupt):
        print(Px,"corrupt")
    else:

        planCT_list,planCTLung_list,bp50_list,bp50Lung_list,itvP_list,itvN_list,gtvP_list, gtvN_list = LookFiles_Snapshot(root+Px)
        print(Px)
        if len(gtvP_list)>0:
            gtvP_np  = nib.load(gtvP_list[0]).get_fdata()
            gtvN_np  = nib.load(gtvP_list[0]).get_fdata()
            bp50_np  = nib.load(bp50_list[0]).get_fdata()

            indicesHR_GTVp_x,indicesHR_GTVp_y,indicesHR_GTVp_z,indicesLR_GTVp_x,indicesLR_GTVp_y,indicesLR_GTVp_z= CountSlidesPerCategory(gtvP_np,GTVpSlide_AvgTresh)
            indicesHR_GTVn_x,indicesHR_GTVn_y,indicesHR_GTVn_z,indicesLR_GTVn_x,indicesLR_GTVn_y,indicesLR_GTVn_z= CountSlidesPerCategory(gtvN_np,GTVnSlide_AvgTresh)
            if False:
                SelectIndexAndScreenshot(Px,"HighRatio_GTVp",indicesHR_GTVp_x,indicesHR_GTVp_y,indicesHR_GTVp_z,bp50_np,gtvP_np)
                SelectIndexAndScreenshot(Px,"LowRatio_GTVp",indicesHR_GTVn_x,indicesHR_GTVn_y,indicesHR_GTVn_z,bp50_np,gtvP_np)
                SelectIndexAndScreenshot(Px,"HighRatio_GTVn",indicesLR_GTVp_x,indicesLR_GTVp_y,indicesLR_GTVp_z,bp50_np,gtvN_np)
                SelectIndexAndScreenshot(Px,"LowRatio_GTVn",indicesLR_GTVn_x,indicesLR_GTVn_y,indicesLR_GTVn_z,bp50_np,gtvN_np)

            totHR_GTVp_x+=len(indicesHR_GTVp_x)
            totHR_GTVp_y+=len(indicesHR_GTVp_y)
            totHR_GTVp_z+=len(indicesHR_GTVp_z)
            totLR_GTVp_x+=len(indicesLR_GTVp_x)
            totLR_GTVp_y+=len(indicesLR_GTVp_y)
            totLR_GTVp_z+=len(indicesLR_GTVp_z)

            totHR_GTVn_x+=len(indicesHR_GTVn_x)
            totHR_GTVn_y+=len(indicesHR_GTVn_y)
            totHR_GTVn_z+=len(indicesHR_GTVn_z)
            totLR_GTVn_x+=len(indicesLR_GTVn_x)
            totLR_GTVn_y+=len(indicesLR_GTVn_y)
            totLR_GTVn_z+=len(indicesLR_GTVn_z)

        if len(itvP_list)>0:
            itvP_np  = nib.load(itvP_list[0]).get_fdata()
            itvN_np  = nib.load(itvN_list[0]).get_fdata()
            avgct_np  = nib.load(planCT_list[0]).get_fdata()

            indicesHR_ITVp_x,indicesHR_ITVp_y,indicesHR_ITVp_z,indicesLR_ITVp_x,indicesLR_ITVp_y,indicesLR_ITVp_z= CountSlidesPerCategory(itvP_np,ITVpSlide_AvgTresh)
            indicesHR_ITVn_x,indicesHR_ITVn_y,indicesHR_ITVn_z,indicesLR_ITVn_x,indicesLR_ITVn_y,indicesLR_ITVn_z= CountSlidesPerCategory(itvN_np,ITVnSlide_AvgTresh)
            if False:
                SelectIndexAndScreenshot(Px,"HighRatio_ITVp",indicesHR_ITVp_x,indicesHR_ITVp_y,indicesHR_ITVp_z,avgct_np,itvP_np)
                SelectIndexAndScreenshot(Px,"LowRatio_ITVp",indicesHR_ITVn_x,indicesHR_ITVn_y,indicesHR_ITVn_z,avgct_np,itvP_np)
                SelectIndexAndScreenshot(Px,"HighRatio_ITVn",indicesLR_ITVp_x,indicesLR_ITVp_y,indicesLR_ITVp_z,avgct_np,itvN_np)
                SelectIndexAndScreenshot(Px,"LowRatio_ITVn",indicesLR_ITVn_x,indicesLR_ITVn_y,indicesLR_ITVn_z,avgct_np,itvN_np)

            totHR_ITVp_x+=len(indicesHR_ITVp_x)
            totHR_ITVp_y+=len(indicesHR_ITVp_y)
            totHR_ITVp_z+=len(indicesHR_ITVp_z)
            totLR_ITVp_x+=len(indicesLR_ITVp_x)
            totLR_ITVp_y+=len(indicesLR_ITVp_y)
            totLR_ITVp_z+=len(indicesLR_ITVp_z)

            totHR_ITVn_x+=len(indicesHR_ITVn_x)
            totHR_ITVn_y+=len(indicesHR_ITVn_y)
            totHR_ITVn_z+=len(indicesHR_ITVn_z)
            totLR_ITVn_x+=len(indicesLR_ITVn_x)
            totLR_ITVn_y+=len(indicesLR_ITVn_y)
            totLR_ITVn_z+=len(indicesLR_ITVn_z)
        count+=1
        if count >5:
            break

    
print("Tot GTVp HR",totHR_GTVp_x,totHR_GTVp_y,totHR_GTVp_z,"LR",totLR_GTVp_x,totLR_GTVp_y,totLR_GTVp_z)
print("Tot GTVn HR",totHR_GTVn_x,totHR_GTVn_y,totHR_GTVn_z,"LR",totLR_GTVn_x,totLR_GTVn_y,totLR_GTVn_z)

print("Tot ITVp HR",totHR_ITVp_x,totHR_ITVp_y,totHR_ITVp_z,"LR: ",totLR_ITVp_x,totLR_ITVp_y,totLR_ITVp_z)
print("Tot ITVn HR",totHR_ITVn_x,totHR_ITVn_y,totHR_ITVn_z,"LR: ",totLR_ITVn_x,totLR_ITVn_y,totLR_ITVn_z)

NameError: name 'PxList' is not defined

In [ ]:
#1007981 - Is incorrect